In [13]:
import numpy as np
import pandas as pd
from datetime import date


In [14]:
df = pd.read_excel('/Users/rohan/Data Science Project/GlassdoorSalaries.xls')

**1. Handling null values**

In [15]:
df.isnull().sum() # Most of the companies do not have any data for competitors. We would get rid of this column
df.drop(columns='Competitors', inplace=True)

# In the scraper, null values have been filled with '-1'. Similarly, the columns which have null values in our data frame
# can be handled by replacing them with -1.

df.fillna(-1, inplace=True)
df.isnull().sum()


Unnamed: 0           0
Job Title            0
Salary Estimate      0
Company Name         0
Location             0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64

**2. Parsing salary column**

In [16]:
# Since we would need salaries of all the positions to perform predictions, companies who do not have any data
# for salary need to be removed.

df[df["Salary Estimate"]==-1]["Salary Estimate"].count() # 216 postings without salary data
df = df[df["Salary Estimate"]!= -1]

# Parsing salary

salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.lower().replace('k','').replace('$',''))

#Segregate per hour salaries and employer provided salary

df["Per hour"] = df["Salary Estimate"].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df["Employer Provided"] = df["Salary Estimate"].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

# Remove 'employer provided salary' and 'per hour' from salary
fin_sal = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

# Calculate minimum, maximum and average salary for each position. Inserting 0 for those position carrying only minimum salary
df['min_sal'] = fin_sal.apply(lambda x: int(x.split('-')[0].strip()) if '-' in x else int(x))
df['max_sal'] = fin_sal.apply(lambda x: int(x.split('-')[0].strip()) if '-' in x else 0)
df['avg_sal'] = (df['min_sal'] + df['max_sal'])/2

**3. Separating company name and ratings**

In [17]:
df['Ratings'] = df['Company Name'].apply(lambda x: float(x.split('\n')[1]) if '\n' in x else -1)
df['Company text'] = df.apply(lambda x: x['Company Name'] if x['Ratings']== -1 else x['Company Name'].split('\n')[0], axis = 1)

**4. Creation of new column for remote jobs and separating city and state**

In [18]:
df["Remote"] = df['Location'].apply(lambda x: 1 if 'remote' in x.lower() else 0)
df['City'] =   df['Location'].apply(lambda x: x.split(',')[0] if ',' in x.lower() else 'NA')
df['State'] =  df['Location'].apply(lambda x: x.split(', ')[1] if ',' in x.lower() else 'NA')

**5. Age calculation of the company**

In [19]:
todays_date = date.today()
df['Age_Company'] = df['Founded'].apply(lambda x: todays_date.year -x if x>-1 else x) 

**6. Grouping size column into greater than 1000 and less than 1000**

In [20]:
df['Size_greater_1000'] = df['Size'].apply(lambda x: -1 if type(x) == int or (type(x)==str and 'unknown' in x.lower()) else 1 if type(x)== str and '1000' in x.lower() else 0)


**7. Drop 1st column**

In [21]:
df.drop(columns='Unnamed: 0', inplace=True)

**8. Converting 'Los Angeles' state to 'CA'**

In [22]:
df['State']= df['State'].replace(['Los Angeles'],'CA')

**9. Exploring job title column**

In [23]:
def seniority(title):
    if "jr" in title.lower() or "junior" in title.lower():
        return "Jr"
    elif "sr" in title.lower() or "senior" in title.lower() or "lead" in title.lower() or "principal" in title.lower():
        return "Sr"
    else:
        return "na"

df["Seniority"]=df["Job Title"].apply(seniority)
df['Seniority'].value_counts()

na    555
Sr    226
Jr      3
Name: Seniority, dtype: int64

**10. Conversion of cleaned data to csv which would be used later for model building**

In [26]:
df.to_csv('Cleaned_data_salary.csv',index=False)